# Voltage Mode Buck with Behaviour Imitation (BI)  


## 📝Revisoin History

- **2025-06-29** Initial release.


## 📁 Directories
```
PyTorch2LTspice/
├── PyTorch2LTspice/
│   └── PyTorch2LTspice.py
└── Example/
    └── BUCK_VM_BI/
        └── 1_train/
            ├── BUCK_VM_BI_gym1.asc             # LTspice schematics
            ├── BUCK_VM_BI_gym2.asc             # LTspice schematics
            ├── *BUCK_VM_BI_param.txt           # Parameter file for LTspice simulation (backup from last LTSpice simulation)
            ├── *BUCK_VM_BI_nn.sp               # Actor subcircuit file (backup from last LTSpice simulation)
            ├── *actor_final.pth                # Actor PyTorch model (backup from last LTSpice simulation)
            └── *gym                            # Working directly for the training
                ├── *log.csv                    # Log file of the simulation
                ├── *loss_plot.html             # Reward plot
                ├── *BUKC_VM_BI_param_epXX.txt  # Parameter file for each episode
                ├── *BUKC_VM_BI_nn_epXX.sp      # Actor subcircuit file for each episode
                ├── *actor_epXX.pth             # Actor model for each episode
                ├── *actor_epXX.pth             # Actor model for each episode
* Files/Directory created by this notebook.
```

## ⚔️ LTspice Training Circuit

### Gym1
![BUCK_VM_BI_gym1.png](.\BUCK_VM_BI_gym1.png)

### Gym2
![BUCK_VM_BI_gym2.png](.\BUCK_VM_BI_gym2.png)

## 📗Import Libraries

In [ ]:
import os
import shutil
import sys
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
from PyLTSpice import SimRunner, RawRead, LTspice
sys.path.insert(0, os.path.join(os.getcwd(), '..', '..', '..', 'PyTorch2LTspice'))
from PyTorch2LTspice import export_model_to_ltspice
from datetime import datetime
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

## ⚙️Configuration

In [ ]:
# Model IO count
NNIN=19
NNOUT=1

In [ ]:
# Hyperparameters
STEPS_PER_EPISODE = 2048    
EPOCH             = 200
SIM_TIMEOUT       = 500   #LTSPICE timeout time (sec)

In [ ]:
# File/Directory
ASCFILE1 = 'BUCK_VM_BI_gym1.asc'
ASCFILE2 = 'BUCK_VM_BI_gym2.asc'
NNFILE = 'BUCK_VM_BI_nn.sp'
PARAMFILE = 'BUCK_VM_BI_param.txt'
WORKDIR = './gym'
MODEL_ACTOR = 'actor_final.pth'
#create WORKDIR if it doesn't exist
os.makedirs(WORKDIR, exist_ok=True)

## 🧩Helping Functions

### Helping function to create parameter file

In [ ]:
def generate_param_file(params, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for name, value in params.items():
            f.write(f".param {name}={value}\n")
        f.write(f".include {NNFILE}\n")
        f.write("X99 NNin1 NNin2 NNin3 NNin4 NNin5 NNin6 NNin7 NNin8 NNin9 NNin10 NNin11 NNin12 NNin13 NNin14 NNin15 NNin16 NNin17 NNin18 NNin19 NNout1 ActorSubckt\n")
        f.write(".save V(ctrlclk) V(NNin1) V(NNin2) V(NNin3) V(NNin4) V(NNin5) V(NNin6) V(NNin7) V(NNin8) V(NNin9) V(NNin10) V(NNin11) V(NNin12) V(NNin13) V(NNin14) V(NNin15) V(NNin16) V(NNin17) V(NNin18) V(NNin19) V(NNout1) V(NNpwm)\n")


Helping functions to create parameter

In [ ]:
def generate_params_random():
    return {
        'vin':  np.random.uniform(150, 250),
        'ro':   np.random.uniform(5, 80),
        'Lo':   np.random.uniform(20e-6, 100e-6),
        'vref': np.random.uniform(50, 150),
        'fsw':  50e3,
        'VMAX': 250,
        'STEPS': STEPS_PER_EPISODE
    }


## 🧩Create Actor Networks
Loads .pth file if MODEL_ACTOR files exists. Otherwise creates new network.

In [ ]:
class Actor(nn.Module):
    def __init__(self, input_dim=19, hidden1=32, hidden2=16, output_dim=1):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden1), nn.ReLU(),
            nn.Linear(hidden1, hidden2), nn.ReLU(),
            nn.Linear(hidden2, output_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)


# Instantiate networks and optimizers
device = torch.device('cpu')
actor  = Actor().to(device)
optimizer  = optim.Adam(actor.parameters(), lr=1e-4)
loss_fn = nn.MSELoss()


# Load saved models if available
if os.path.exists(MODEL_ACTOR):
    #actor.load_state_dict(torch.load(MODEL_ACTOR, map_location=device, weights_only=False))
    actor.load_state_dict(torch.load(MODEL_ACTOR, map_location=device))
    print(f"Loaded saved actor model from {MODEL_ACTOR}")
else:
    torch.save(actor.state_dict(), MODEL_ACTOR)
    print(f"Created new actor model and saved to {MODEL_ACTOR}")

## 🧩LTSpice execution routine 

Helping function to extract Status/Action data from .RAW file.
Stops data extraction once duty output gets out of range. 

In [ ]:
def extract_data(df, clk_col='V(ctrlclk)', threshold=0.5):
    clk = df[clk_col].values

    # Check if the clock starts at high level
    if clk[0] > threshold:
        raise ValueError("Clock started with Level Hi")

    indices = []
    state = 'LOW'

    for i in range(1, len(clk)):
        if state == 'LOW' and clk[i - 1] <= threshold and clk[i] > threshold:
            state = 'HIGH'  # Rising edge detected
        elif state == 'HIGH' and clk[i - 1] > threshold and clk[i] <= threshold:
            # Falling edge detected
            indices.append(i)
            state = 'LOW'
    df_falling_edges = df.iloc[indices].reset_index(drop=True)
    return df_falling_edges

LTspice execution

In [ ]:
def run_episode(asc_file, work_dir):
    # 1) Create PyLTspice SimRunner instance
    runner = SimRunner(output_folder=work_dir, simulator=LTspice)
    netlist = runner.create_netlist(asc_file)
    
    # 2) Run simulation
    raw, log = runner.run_now(netlist, timeout=SIM_TIMEOUT)
    raw_data = RawRead(raw)
    df = raw_data.to_dataframe()
    df = extract_data(df)

    # 3) Extract states, actions
    x_data  = df[[f'V(nnin{i+1})' for i in range(19)]].values[:-1]
    y_data = df['V(nnpwm)'].values[:-1]
    y_pred = df['V(nnout1)'].values[:-1]

    # 4) Crean PyLTspice files
    runner.cleanup_files()

    return x_data, y_data, y_pred, df

## 🧩BI Update Step

In [ ]:
def bi_update(x_data, y_data, epochs=200):
    
    x_tensor  = torch.tensor(x_data,  dtype=torch.float32, device=device)
    y_tensor = torch.tensor(y_data, dtype=torch.float32, device=device).reshape(-1, 1)

    losses = []
    for epoch in range(epochs):
        optimizer.zero_grad()
        y_pred = actor(x_tensor)
        loss = loss_fn(y_pred, y_tensor)
        loss.backward()
        optimizer.step()
        
        losses.append(loss.item())
        if epoch % 20 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item():.6f}")

    return (loss.item())

## 📉Training Status Plot 

In [ ]:
fig_loss = go.FigureWidget()
fig_loss.add_trace(go.Scatter(x=[], y=[], mode='lines+markers', name='loss actor', yaxis='y1'))
fig_loss.update_layout(xaxis=dict(title='Episode'), yaxis=dict(title='loss',type='log'), legend=dict(x=0, y=1.2, orientation='h'))

fig_nn = go.FigureWidget()
fig_nn.add_trace(go.Scatter(x=[], y=[], name='nnin1(Vo/Vmax)', yaxis='y1', mode='lines+markers'))
fig_nn.add_trace(go.Scatter(x=[], y=[], name='nnin17(Vref/Vmax)', yaxis='y1', mode='lines'))
fig_nn.add_trace(go.Scatter(x=[], y=[], name='nnpwm', yaxis='y1', mode='lines+markers'))
fig_nn.add_trace(go.Scatter(x=[], y=[], name='nnout1', yaxis='y1', mode='lines+markers'))
fig_nn.update_layout(xaxis=dict(title='Step'), yaxis=dict(title='NNIO'))
fig_nn.update_layout(legend=dict(orientation="h", x=0.5, y=-0.3, xanchor='center', yanchor='top'),height=500)
t = list(range(STEPS_PER_EPISODE)) 
for i in range(4):
    fig_nn.data[i].x = t



## ♻️Training Loop

In [ ]:
#reload or newly create log.csv
if os.path.exists("./gym/log.csv"):
    summary_df = pd.read_csv("./gym/log.csv") 
else:
    summary_df = pd.DataFrame(columns=['episode','sim time','loss_actor','vin','ro','Lo','vref','fsw','std','steps','epoch','Vout/Vin','Iout/Icrit'])


def train_loop(base_ep, num_ep, param_fn, asc_file):        
    # Remove episode data from summary_df from base_ep onwards
    global summary_df
    summary_df = summary_df[summary_df['episode'] < base_ep].reset_index(drop=True)

    # Main loop
    for ep in range(base_ep , base_ep+num_ep):
        try:
            sim_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print(f">>> {sim_time}")

            # 1) Save actor files
            torch.save(actor.state_dict(), WORKDIR+f"/actor_ep{ep}.pth")
            torch.save(actor.state_dict(), MODEL_ACTOR)

            # 2) Export current actor to SPICE subckt
            export_model_to_ltspice(actor.model, filename=NNFILE, subckt_name='ActorSubckt', verbose=False)
            shutil.copy2(NNFILE, WORKDIR)
            name, ext = os.path.splitext(NNFILE)   
            shutil.copy2(NNFILE, f"{WORKDIR}/{name}_ep{ep}{ext}")   
            
            # 3) Generate Parameter file
            params = param_fn()
            generate_param_file(params, PARAMFILE)
            shutil.copy2(PARAMFILE, WORKDIR)
            name, ext = os.path.splitext(PARAMFILE)   
            shutil.copy2(PARAMFILE, f"{WORKDIR}/{name}_ep{ep}{ext}")   
            
            # 4) Run one full episode in LTspice
            vout_vin = params['vref'] / params['vin']
            iout_icrit = params['vref'] / params['ro'] / (0.5 * params['vref'] * (params['vin'] - params['vref']) / params['fsw'] / 0.0002 / params['vin'])
            print(f"[Ep{ep}/{base_ep+num_ep-1}] Vout/Vin:{vout_vin:.2f}, Iout/Icrit:{iout_icrit:.3f}")   #L=200uH
            x_data, y_data, y_pred, df = run_episode(asc_file, WORKDIR)

            # 5) Perform 
            loss_actor = bi_update(x_data, y_data, epochs=EPOCH)
            
            # 8) Update&Save episode graph
            fig_nn.data[0].y = df['V(nnin1)']
            fig_nn.data[1].y = df['V(nnin17)']
            fig_nn.data[2].y = df['V(nnpwm)']
            fig_nn.data[3].y = df['V(nnout1)']
            fig_nn.update_layout(title_text=f"Ep{ep}: Vout/Vin={vout_vin:.2f}, Iout/Icrit={iout_icrit:.2f}")

            # 9) Append summary
            summary_df.loc[len(summary_df)] = {
                'episode':      ep,
                'sim time':     sim_time,
                'loss_actor':   loss_actor,
                'vin':          params['vin'],
                'ro':           params['ro'],
                'Lo':           params['Lo'],
                'vref':         params['vref'],
                'fsw':          params['fsw'],
                'steps':        STEPS_PER_EPISODE,
                'epoch':        EPOCH,
                'Vout/Vin':     vout_vin,
                'Iout/Icrit':   iout_icrit
            }

            # 10) Update/Save learning curve plot
            fig_loss.data[0].x = summary_df['episode']
            fig_loss.data[0].y = summary_df['loss_actor']
            loss_html_path = os.path.join(WORKDIR, "loss_plot.html")
            fig_loss.write_html(loss_html_path, include_plotlyjs='cdn')


            # 12) Save summary to CSV
            episode_csv = os.path.join(WORKDIR, 'log.csv')
            summary_df.to_csv(episode_csv, index=False)    

            # 13) Save actor files
            torch.save(actor.state_dict(), MODEL_ACTOR)

        except Exception as e:
            print(f"[Ep{ep}/{base_ep+num_ep-1}] ERROR: Failed with exception: {e}")
            continue



##  🧠Training Display

In [ ]:
display(fig_loss)
display(fig_nn)

##  🧠Training Steps(Example)

In [ ]:
train_loop(1,300,generate_params_random,ASCFILE1)

In [ ]:
train_loop(201,200,generate_params_random,ASCFILE2)